# Kanana RAG Fine-tuning Notebook

This notebook fine-tunes the Kanana 8B instruct model on RAG tasks using the Jecheon tourism dataset.

**Model:** kakaocorp/kanana-1.5-8b-instruct-2505

**Training Data Format:**
```
[Instruction]
당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

Information:
{content1}
Information:
{content2}
Question: {question}
```

**Evaluation Metrics:**

*Automatic Metrics (during training):*
- **Perplexity**: Model confidence (lower is better)
- **Exact Match (EM)**: Exact answer matching accuracy
- **ROUGE-L**: Text overlap score
- **BERTScore**: Semantic similarity with Korean BERT
- **Token F1**: Token-level overlap

*LLM-as-a-Judge Metrics (post-training):*
- **Accuracy** (true/false): No hallucinations or extra details
- **Helpfulness** (1-10): How helpful is the answer
- **Relevance** (1-10): Does it fully address the question
- **Depth** (1-10): Level of detail in the response

All metrics are tracked in Weights & Biases for each training run.

In [ ]:
# Install required packages
!pip install transformers peft datasets wandb bitsandbytes accelerate evaluate rouge_score bert_score anthropic

In [ ]:
# Check CUDA availability
import os
import torch

os.environ["NVIDIA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Login to Weights & Biases
import wandb

# Login to wandb (you'll be prompted for your API key)
wandb.login()

# Initialize wandb project
wandb.init(
    project="kanana-rag-finetuning",
    name="kanana-1.5-8b-instruct-rag",
    config={
        "model": "kakaocorp/kanana-1.5-8b-instruct-2505",
        "task": "RAG fine-tuning",
        "dataset": "Jecheon Tourism"
    }
)

In [ ]:
# Load base model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-1.5-8b-instruct-2505"

print(f"Loading model: {model_name}")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully!")

In [ ]:
# Load RAG training data
import json
from datasets import Dataset

data_path = "/home/user/goodganglabs/data/processed/training_data.jsonl"

# Load JSONL data
data_list = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

print(f"Loaded {len(data_list)} training examples")
print("\nFirst example:")
print(json.dumps(data_list[0], ensure_ascii=False, indent=2)[:500])

In [ ]:
# Convert to requested RAG format with instruction
def format_rag_data(example):
    """Convert RAG data to the requested format with instruction:
    [Instruction]
    당신은 제천시 관광 안내 전문가입니다...
    
    Information:
    {content1}
    Information:
    {content2}
    Question: {question}
    """
    instruction = """당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

답변 시 주의사항:
1. 관련 문서의 내용만을 바탕으로 답변하세요
2. 문서에 정보가 없으면 "제공된 정보에는 해당 내용이 없습니다"라고 답변하세요
3. 추측하거나 문서 외부 지식을 사용하지 마세요
4. 간결하고 이해하기 쉽게 답변하세요"""
    
    documents = example['documents']
    question = example['question']
    answer = example['answer']
    
    # Build information sections
    info_sections = []
    for doc in documents:
        info_sections.append(f"Information:\n{doc['content']}")
    
    # Combine: instruction + information sections + question
    prompt = instruction + "\n\n"
    prompt += "\n\n".join(info_sections)
    prompt += f"\n\nQuestion: {question}"
    
    return {
        "prompt": prompt,
        "answer": answer
    }

# Apply formatting
formatted_data = []
for example in data_list:
    formatted_data.append(format_rag_data(example))

print(f"Formatted {len(formatted_data)} examples")
print("\nExample formatted prompt:")
print(formatted_data[0]['prompt'][:500])
print(f"\nAnswer: {formatted_data[0]['answer']}")

In [ ]:
# Create training dataset with proper format
def formatting_prompts_func(examples):
    """Format prompts for training"""
    prompts = examples["prompt"]
    answers = examples["answer"]
    
    EOS_TOKEN = tokenizer.eos_token
    
    texts = []
    for prompt, answer in zip(prompts, answers):
        # Combine prompt and answer with EOS token
        text = f"{prompt}\n\nAnswer: {answer}{EOS_TOKEN}"
        texts.append(text)
    
    return {"text": texts}

# Create dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"Dataset size: {len(dataset)}")
print(f"\nDataset features: {dataset.features}")
print(f"\nFirst training example:")
print(dataset[0]['text'][:500])

In [ ]:
# Tokenize dataset
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"], 
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_tensors="pt"
    )
    tokens["labels"] = tokens["input_ids"].clone()
    return tokens

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=["text", "prompt", "answer"]
)

print(f"Tokenized dataset size: {len(tokenized_dataset)}")
print(f"Features: {tokenized_dataset.features}")

In [ ]:
# Define compute_metrics function for evaluation
def compute_metrics(eval_preds):
    """
    Compute evaluation metrics during training.
    
    Metrics:
    - Perplexity: Language modeling quality
    - EM (Exact Match): Exact answer matching accuracy
    - ROUGE-L: Text overlap (longest common subsequence)
    - BERTScore: Semantic similarity (Korean model)
    - F1 Score: Token-level overlap
    """
    predictions, labels = eval_preds
    
    # Decode predictions and labels
    # Replace -100 in labels (padding tokens)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Clean up whitespace
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    
    # 1. Exact Match (EM)
    def normalize_text(text):
        """Normalize text for EM calculation"""
        import re
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text.strip())
        # Lowercase (optional, adjust based on needs)
        # text = text.lower()
        return text
    
    exact_matches = [
        1 if normalize_text(pred) == normalize_text(label) else 0
        for pred, label in zip(decoded_preds, decoded_labels)
    ]
    em_score = np.mean(exact_matches)
    
    # 2. ROUGE Score
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        rouge_types=["rougeL"]
    )
    
    # 3. BERTScore (Korean model)
    bertscore_result = bertscore_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        lang="ko",
        model_type="klue/roberta-base",  # Korean BERT model
        device="cuda" if torch.cuda.is_available() else "cpu"
    )
    
    # 4. F1 Score (token-level)
    def f1_score(pred, label):
        pred_tokens = set(pred.split())
        label_tokens = set(label.split())
        
        if len(pred_tokens) == 0 or len(label_tokens) == 0:
            return 0.0
        
        common = pred_tokens & label_tokens
        if len(common) == 0:
            return 0.0
        
        precision = len(common) / len(pred_tokens)
        recall = len(common) / len(label_tokens)
        
        return 2 * (precision * recall) / (precision + recall)
    
    f1_scores = [f1_score(pred, label) for pred, label in zip(decoded_preds, decoded_labels)]
    avg_f1 = np.mean(f1_scores)
    
    # Compile results
    result = {
        "exact_match": em_score,
        "rouge_l": rouge_result["rougeL"],
        "bertscore_precision": np.mean(bertscore_result["precision"]),
        "bertscore_recall": np.mean(bertscore_result["recall"]),
        "bertscore_f1": np.mean(bertscore_result["f1"]),
        "token_f1": avg_f1,
    }
    
    # Note: Perplexity is automatically computed from eval_loss by Trainer
    
    return result

print("compute_metrics function defined!")
print("Metrics will be logged to wandb during training.")

In [ ]:
# Load evaluation metrics
from evaluate import load
import numpy as np
import torch

# Load metrics
rouge = load('rouge')
bertscore_metric = load('bertscore')

print("Loaded evaluation metrics:")
print("  - Exact Match (EM)")
print("  - ROUGE (ROUGE-L)")
print("  - BERTScore (Korean semantic similarity)")
print("  - F1 Score (token overlap)")
print("  - Perplexity (from loss)")

In [ ]:
# Split dataset into train and validation (80/20 split)
from datasets import DatasetDict

# Split tokenized dataset
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=1234)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")
print(f"Train/Eval ratio: {len(train_dataset)/len(eval_dataset):.2f}")

# Keep reference to original formatted data for metric computation
formatted_data_dict = {
    "train": formatted_data[:len(train_dataset)],
    "eval": formatted_data[len(train_dataset):]
}

In [ ]:
# Configure LoRA for parameter-efficient fine-tuning
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,                    # LoRA rank
    lora_alpha=32,          # LoRA alpha
    lora_dropout=0.1,       # Dropout probability
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj"
    ]
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Log LoRA config to wandb
wandb.config.update({
    "lora_r": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "lora_dropout": lora_config.lora_dropout,
    "target_modules": lora_config.target_modules
})

In [ ]:
# Check GPU memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Log to wandb
wandb.config.update({
    "gpu_name": gpu_stats.name,
    "gpu_max_memory_gb": max_memory,
    "gpu_start_memory_gb": start_gpu_memory
})

In [ ]:
# Configure training arguments with evaluation and wandb integration
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    # Output settings
    output_dir="./outputs/kanana-rag",
    overwrite_output_dir=True,
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=10,
    
    # Optimization
    bf16=True,
    optim="adamw_torch",
    lr_scheduler_type="linear",
    
    # Logging
    logging_steps=5,
    logging_dir="./logs",
    report_to="wandb",  # Enable wandb reporting
    
    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,  # Load best model at end
    
    # Evaluation with metrics
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=25,  # Evaluate every 25 steps
    metric_for_best_model="bertscore_f1",  # Use BERTScore F1 for best model
    greater_is_better=True,  # Higher BERTScore is better
    
    # Other
    seed=1234,
    data_seed=1234,
    remove_unused_columns=True,
    predict_with_generate=False,  # Use logits for faster evaluation
)

print("Training arguments configured:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Evaluation strategy: {training_args.evaluation_strategy} (every {training_args.eval_steps} steps)")
print(f"  Best model metric: {training_args.metric_for_best_model}")
print(f"\nMetrics tracked:")
print(f"  - Perplexity (from eval_loss)")
print(f"  - Exact Match (EM)")
print(f"  - ROUGE-L")
print(f"  - BERTScore (P/R/F1)")
print(f"  - Token F1")
print(f"\nAll metrics will be logged to WandB!")

In [ ]:
# Initialize Trainer with evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

print("Trainer initialized successfully!")
print(f"\nTraining setup:")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Eval samples: {len(eval_dataset)}")
print(f"  Metrics: Perplexity, EM, ROUGE-L, BERTScore, Token F1")
print(f"  Best model selection: {training_args.metric_for_best_model}")
print(f"\n⚠️ Note: BERTScore evaluation may take some time due to Korean BERT model inference.")

In [ ]:
# Start training
print("Starting training...")
print("=" * 50)

trainer_stats = trainer.train()

print("\n" + "=" * 50)
print("Training completed!")
print(f"Training loss: {trainer_stats.training_loss:.4f}")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f}s")

In [ ]:
# Check final GPU memory usage
final_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nFinal GPU memory: {final_gpu_memory} GB")
print(f"Peak memory used: {final_gpu_memory - start_gpu_memory} GB")

# Log final stats to wandb
wandb.log({
    "final_gpu_memory_gb": final_gpu_memory,
    "peak_memory_used_gb": final_gpu_memory - start_gpu_memory
})

In [ ]:
# Save the fine-tuned model
output_dir = "./outputs/kanana-rag-final"

print(f"Saving model to {output_dir}...")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved successfully!")
print(f"\nModel location: {output_dir}")

In [ ]:
# Test inference with a sample
print("Testing inference...\n")

# Get a test example
test_prompt = formatted_data[0]['prompt']
expected_answer = formatted_data[0]['answer']

print("Test Prompt:")
print(test_prompt[:300])
print("\n...")

# Tokenize and generate
model.eval()
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nGenerated Response:")
print(generated_text[len(test_prompt):])

print("\nExpected Answer:")
print(expected_answer)

In [ ]:
# Close wandb run
wandb.finish()
print("WandB run finished!")

In [ ]:
# EXAMPLE: Run LLM-as-a-Judge evaluation on test samples
# Uncomment and run this cell after training is complete

"""
# Step 1: Set your Anthropic API key
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    print("⚠️  Please set ANTHROPIC_API_KEY environment variable")
    print("   export ANTHROPIC_API_KEY='your-api-key'")
else:
    # Step 2: Generate responses for evaluation (10 samples)
    print("Generating responses for evaluation...")
    eval_samples = generate_responses_for_evaluation(
        model=trainer.model,
        tokenizer=tokenizer,
        test_samples=formatted_data,
        max_samples=10
    )
    
    # Step 3: Evaluate with LLM judge
    print("\nEvaluating with Claude as judge...")
    judge_results = evaluate_batch_with_llm_judge(
        samples=eval_samples,
        api_key=api_key,
        judge_model="claude-3-5-sonnet-20241022"
    )
    
    # Step 4: Log results to WandB
    print("\nLogging results to WandB...")
    metrics = log_llm_judge_to_wandb(judge_results)
    
    # Step 5: Save results to file
    output_path = "./outputs/llm_judge_results.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(judge_results, f, ensure_ascii=False, indent=2)
    print(f"\n💾 Results saved to: {output_path}")
"""

print("LLM-as-a-Judge evaluation example loaded!")
print("\nUncomment the code above to run evaluation after training.")

In [ ]:
# Analyze and log LLM judge results to WandB

def analyze_llm_judge_results(results: List[Dict]) -> Dict:
    """
    Analyze LLM judge evaluation results
    
    Returns:
        Dictionary with aggregated metrics
    """
    # Extract scores
    accuracy_scores = []
    helpfulness_scores = []
    relevance_scores = []
    depth_scores = []
    
    for result in results:
        # Accuracy (true/false → 1/0)
        if 'accuracy' in result and 'accuracy' in result['accuracy']:
            acc = result['accuracy']['accuracy']
            if isinstance(acc, bool):
                accuracy_scores.append(1 if acc else 0)
            elif isinstance(acc, str):
                accuracy_scores.append(1 if acc.lower() == 'true' else 0)
        
        # Helpfulness (1-10)
        if 'helpfulness' in result and 'helpfulness' in result['helpfulness']:
            helpfulness_scores.append(int(result['helpfulness']['helpfulness']))
        
        # Relevance (1-10)
        if 'relevance' in result and 'relevance' in result['relevance']:
            relevance_scores.append(int(result['relevance']['relevance']))
        
        # Depth (1-10)
        if 'depth' in result and 'depth' in result['depth']:
            depth_scores.append(int(result['depth']['depth']))
    
    # Calculate averages
    metrics = {
        "llm_judge_accuracy": np.mean(accuracy_scores) if accuracy_scores else 0,
        "llm_judge_helpfulness": np.mean(helpfulness_scores) if helpfulness_scores else 0,
        "llm_judge_relevance": np.mean(relevance_scores) if relevance_scores else 0,
        "llm_judge_depth": np.mean(depth_scores) if depth_scores else 0,
        "num_samples_evaluated": len(results)
    }
    
    return metrics

def log_llm_judge_to_wandb(results: List[Dict], run_name: str = None):
    """
    Log LLM judge evaluation results to WandB
    
    Args:
        results: LLM judge evaluation results
        run_name: Optional run name for WandB
    """
    # Analyze results
    metrics = analyze_llm_judge_results(results)
    
    # Log to WandB
    wandb.log({
        "eval/llm_judge_accuracy": metrics["llm_judge_accuracy"],
        "eval/llm_judge_helpfulness": metrics["llm_judge_helpfulness"],
        "eval/llm_judge_relevance": metrics["llm_judge_relevance"],
        "eval/llm_judge_depth": metrics["llm_judge_depth"],
    })
    
    # Create summary table
    summary_table = wandb.Table(
        columns=["Metric", "Score", "Scale"],
        data=[
            ["Accuracy", f"{metrics['llm_judge_accuracy']:.2%}", "0-100%"],
            ["Helpfulness", f"{metrics['llm_judge_helpfulness']:.1f}", "1-10"],
            ["Relevance", f"{metrics['llm_judge_relevance']:.1f}", "1-10"],
            ["Depth", f"{metrics['llm_judge_depth']:.1f}", "1-10"],
        ]
    )
    wandb.log({"llm_judge_summary": summary_table})
    
    # Create detailed results table
    detailed_data = []
    for i, result in enumerate(results[:20]):  # First 20 samples
        detailed_data.append([
            i + 1,
            result.get('question', '')[:50] + "...",
            result.get('response', '')[:100] + "...",
            result.get('accuracy', {}).get('accuracy', 'N/A'),
            result.get('helpfulness', {}).get('helpfulness', 'N/A'),
            result.get('relevance', {}).get('relevance', 'N/A'),
            result.get('depth', {}).get('depth', 'N/A'),
        ])
    
    detailed_table = wandb.Table(
        columns=["#", "Question", "Response", "Accuracy", "Helpfulness", "Relevance", "Depth"],
        data=detailed_data
    )
    wandb.log({"llm_judge_detailed_results": detailed_table})
    
    print("\n" + "="*60)
    print("LLM Judge Evaluation Results")
    print("="*60)
    print(f"Accuracy:    {metrics['llm_judge_accuracy']:.2%} (no hallucinations)")
    print(f"Helpfulness: {metrics['llm_judge_helpfulness']:.1f}/10")
    print(f"Relevance:   {metrics['llm_judge_relevance']:.1f}/10")
    print(f"Depth:       {metrics['llm_judge_depth']:.1f}/10")
    print(f"\nEvaluated {metrics['num_samples_evaluated']} samples")
    print("="*60)
    
    return metrics

print("LLM judge analysis and logging functions loaded!")
print("\nTo use:")
print("  metrics = log_llm_judge_to_wandb(results)")
print("\nThis will log all 4 metrics to WandB dashboard")

In [ ]:
# Example: Evaluate test samples with LLM-as-a-Judge
# This cell demonstrates how to use LLM judge evaluation

def generate_responses_for_evaluation(
    model,
    tokenizer,
    test_samples: List[Dict],
    max_samples: int = 10
) -> List[Dict]:
    """
    Generate responses for test samples
    
    Args:
        model: Fine-tuned model
        tokenizer: Tokenizer
        test_samples: List of test samples with 'prompt', 'answer', 'documents'
        max_samples: Maximum number of samples to evaluate
        
    Returns:
        List of samples ready for LLM judge evaluation
    """
    model.eval()
    evaluation_samples = []
    
    for i, sample in enumerate(test_samples[:max_samples]):
        print(f"Generating response {i+1}/{min(max_samples, len(test_samples))}...")
        
        # Get the prompt (instruction + information + question)
        prompt = sample['prompt']
        
        # Get the correct context (first document marked as correct)
        context = ""
        for doc in sample.get('documents', []):
            if doc.get('is_correct', False):
                context = doc['content']
                break
        
        # If no correct document found, use first document
        if not context and sample.get('documents'):
            context = sample['documents'][0]['content']
        
        # Generate response
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Extract only the answer part (after "Answer:")
        if "Answer:" in generated_text:
            response = generated_text.split("Answer:")[-1].strip()
        else:
            response = generated_text[len(prompt):].strip()
        
        # Extract question from prompt
        if "Question:" in prompt:
            question = prompt.split("Question:")[-1].strip()
        else:
            question = "Unknown question"
        
        evaluation_samples.append({
            "question": question,
            "context": context,
            "response": response,
            "expected_answer": sample.get('answer', ''),
            "filename": sample.get('documents', [{}])[0].get('filename', 'document')
        })
    
    return evaluation_samples

print("Response generation function loaded!")
print("\nTo use:")
print("  1. Generate responses: samples = generate_responses_for_evaluation(model, tokenizer, formatted_data)")
print("  2. Evaluate with judge: results = evaluate_batch_with_llm_judge(samples, api_key)")

In [ ]:
# LLM-as-a-Judge Evaluation
# Using GPT-4 or Claude to evaluate model responses on 4 metrics
# Based on the paper's evaluation methodology

import json
from typing import Dict, List, Tuple
import anthropic
import os

# Evaluation prompts from the paper
EVALUATION_PROMPTS = {
    "accuracy": """Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below, based solely on a piece of information extracted from a file provided below. Your evaluation should consider the accuracy of the response.

You will check whether the response contains extra details not found in the piece of information provided. If extra details are found, accuracy is false. Otherwise, accuracy is true. Take note that if the response partially addresses the question, but did not provide extra details not found in the piece of information provided, the response will still be considered accurate (hence accuracy = true).

Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the accuracy with true or false by strictly following this JSON format:
{
"accuracy_explanation": <provide an explanation on accuracy, whether extra details outside the content were found.>,
"accuracy": <true/false>
}""",
    
    "helpfulness": """Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below, based solely on a piece of information extracted from a file provided below. Your evaluation should consider the helpfulness of the response.

You will check whether the AI assistant is helpful in answering the question based on the response.

Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the helpfulness on a scale of 1 to 10 by strictly following this JSON format:
{
"helpfulness_explanation": <provide an explanation on helpfulness>,
"helpfulness": <score>
}""",
    
    "relevance": """Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below, based solely on a piece of information extracted from a file provided below. Your evaluation should consider the relevance of the response.

You will check the relevance of the response by evaluating whether the response fully addresses the question.

Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the relevance on a scale of 1 to 10 by strictly following this JSON format:
{
"relevance_explanation": <provide an explanation on relevance>,
"relevance": <score>
}""",
    
    "depth": """Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below, based solely on a piece of information extracted from a file provided below. Your evaluation should consider the depth of the response.

You will check the depth of the response by evaluating the level of detail of the response in answering the question.

Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the depth on a scale of 1 to 10 by strictly following this JSON format:
{
"depth_explanation": <provide an explanation on depth>,
"depth": <score>
}"""
}

def create_user_message(question: str, context: str, response: str, filename: str = "document") -> str:
    """Create user message for LLM judge evaluation"""
    return f"""[The Start of Provided Information Extracted from a File]
Filename: {filename}
Information: {context}
[The End of Provided Information]

[Question]
{question}

[The Start of Assistant's Response]
{response}
[The End of Assistant's Response]"""

def evaluate_with_llm_judge(
    question: str,
    context: str,
    response: str,
    metric: str,
    client,
    model: str = "claude-3-5-sonnet-20241022"
) -> Dict:
    """
    Evaluate a single response using LLM-as-a-Judge
    
    Args:
        question: The question asked
        context: The relevant document context
        response: Model's response to evaluate
        metric: One of 'accuracy', 'helpfulness', 'relevance', 'depth'
        client: Anthropic client instance
        model: Judge model to use
        
    Returns:
        Dictionary with evaluation results
    """
    system_prompt = EVALUATION_PROMPTS[metric]
    user_message = create_user_message(question, context, response)
    
    # Call Claude as judge
    message = client.messages.create(
        model=model,
        max_tokens=1024,
        system=system_prompt,
        messages=[
            {"role": "user", "content": user_message}
        ]
    )
    
    # Parse JSON response
    try:
        result = json.loads(message.content[0].text)
        return result
    except json.JSONDecodeError:
        # Fallback: extract JSON from text
        text = message.content[0].text
        # Try to find JSON in the response
        import re
        json_match = re.search(r'\{[^}]+\}', text, re.DOTALL)
        if json_match:
            return json.loads(json_match.group())
        else:
            return {"error": "Failed to parse JSON", "raw_response": text}

def evaluate_batch_with_llm_judge(
    samples: List[Dict],
    api_key: str,
    judge_model: str = "claude-3-5-sonnet-20241022"
) -> List[Dict]:
    """
    Evaluate multiple samples with LLM judge
    
    Args:
        samples: List of dicts with 'question', 'context', 'response'
        api_key: Anthropic API key
        judge_model: Claude model to use as judge
        
    Returns:
        List of evaluation results
    """
    client = anthropic.Anthropic(api_key=api_key)
    
    results = []
    metrics = ['accuracy', 'helpfulness', 'relevance', 'depth']
    
    for i, sample in enumerate(samples):
        print(f"\nEvaluating sample {i+1}/{len(samples)}...")
        sample_result = {
            "question": sample["question"],
            "context": sample["context"],
            "response": sample["response"]
        }
        
        # Evaluate on all 4 metrics
        for metric in metrics:
            print(f"  - {metric}...", end=" ")
            try:
                eval_result = evaluate_with_llm_judge(
                    question=sample["question"],
                    context=sample["context"],
                    response=sample["response"],
                    metric=metric,
                    client=client,
                    model=judge_model
                )
                sample_result[metric] = eval_result
                print("✓")
            except Exception as e:
                print(f"✗ Error: {e}")
                sample_result[metric] = {"error": str(e)}
        
        results.append(sample_result)
    
    return results

print("LLM-as-a-Judge evaluation functions loaded!")
print("\nAvailable metrics:")
print("  1. Accuracy (true/false) - No hallucinations")
print("  2. Helpfulness (1-10) - How helpful is the answer")
print("  3. Relevance (1-10) - Does it address the question")
print("  4. Depth (1-10) - Level of detail provided")
print("\nUsage:")
print("  Set ANTHROPIC_API_KEY environment variable")
print("  Then run evaluate_batch_with_llm_judge(samples, api_key)")

## Summary

This notebook:
1. ✅ Loads Kanana 1.5 8B instruct model (kakaocorp/kanana-1.5-8b-instruct-2505)
2. ✅ Formats RAG training data with instruction and information format
3. ✅ Splits data into train/validation (80/20)
4. ✅ Applies LoRA for efficient fine-tuning
5. ✅ **Automatic evaluation metrics (during training):**
   - **Perplexity**: Language modeling quality (from eval_loss)
   - **Exact Match (EM)**: Exact answer matching accuracy
   - **ROUGE-L**: Text overlap with longest common subsequence
   - **BERTScore**: Semantic similarity using Korean BERT (klue/roberta-base)
   - **Token F1**: Token-level overlap score
6. ✅ **LLM-as-a-Judge evaluation (post-training):**
   - **Accuracy** (true/false): No hallucinations or extra details
   - **Helpfulness** (1-10): How helpful is the answer
   - **Relevance** (1-10): Does it fully address the question
   - **Depth** (1-10): Level of detail in the response
7. ✅ Tracks all metrics in Weights & Biases
8. ✅ Saves best model based on BERTScore F1
9. ✅ Tests inference on sample data

### Key Features:
- **System instruction** guides the model to be a Jecheon tourism expert
- Model learns to find relevant documents and answer based on them
- Model learns to say "no information available" when appropriate
- **Semantic evaluation** with BERTScore catches paraphrased answers
- **LLM-as-a-Judge** provides qualitative assessment using Claude
- **WandB integration** for real-time metric tracking across runs
- Training format prevents hallucination

### Automatic Evaluation Strategy:
- Evaluate every 25 steps during training
- Track 7 metrics: eval_loss, perplexity, exact_match, rouge_l, bertscore_f1, bertscore_precision, bertscore_recall, token_f1
- Best model selected by BERTScore F1 (semantic similarity)
- All metrics logged to WandB for comparison across runs

### LLM-as-a-Judge Evaluation:
- Run after training on test samples
- Uses Claude 3.5 Sonnet as impartial judge
- Evaluates on 4 dimensions based on research paper methodology
- Results logged to WandB with summary and detailed tables
- Provides qualitative insights beyond automatic metrics

### Usage:
1. **Training**: Run cells sequentially to train the model
2. **LLM Judge**: Uncomment and run LLM judge evaluation cells
3. **Set API key**: `export ANTHROPIC_API_KEY='your-key'`
4. **View results**: Check WandB dashboard for all metrics

### Next Steps:
- Run full evaluation on test set with both automatic and LLM judge metrics
- Compare baseline vs fine-tuned performance
- Upload model to Hugging Face Hub
- Generate comprehensive report with quantitative and qualitative analysis